In [66]:
from prep_data import Data_prepare
import numpy as np 
import os 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [67]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
# index = [11,18,36]
df = dp.combine_male_female(m_file,f_file)
# df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

#### Normalize x with standardScaler

In [68]:
from sklearn.preprocessing import StandardScaler
def normalize(train,test):
    new_x = np.append(train,test,axis=0)
    scaler = StandardScaler()
    scaler.fit(new_x)
    train = scaler.transform(train)
    test= scaler.transform(test)
    return train, test

In [69]:
## reshape x_train and x_test to 3D for LSTM 
def reshape_x(x_train,x_test,feature_num):
    x_train = np.reshape(x_train, (x_train.shape[0],1,feature_num))
    x_test = np.reshape(x_test, (x_test.shape[0],1,feature_num))
    return x_train,x_test

In [70]:
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        # print(new_x.shape)
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    return new_x,new_y

In [71]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [72]:
temp1, temp2 = get_x_y_train(x,y)
print(temp1.shape,temp2.shape)

(1464, 2048) (1464, 1)


#### LSTM model 

In [76]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.optimizers import SGD

def lstm_model():
    tf.keras.backend.clear_session()

    model = Sequential()
    model.add(LSTM(512, input_shape=(1,2048),activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(16,activation="relu"))
    model.add(Dropout(0.2))
    # model.add(LSTM(50))
    # model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    return model 

In [77]:
## leave one out 
# print(new_x.shape,new_y.shape)
all_accuracy=[]
index = 0
for d in range(x.shape[0]):
    print("leave one out # person:",d)
    if d == 0:
        x_train,y_train = get_x_y_train(x[d+1:],y[d+1:])
        x_test,y_test =x[d],y[d]
    elif d == x.shape[0]-1:
        x_train,y_train = get_x_y_train(x[:d],y[:d])
        x_test,y_test = x[d],y[d]
    else:
        x_train,y_train = get_x_y_train(np.append(x[:d],x[d+1:],axis=0),np.append(y[:d],y[d+1:],axis=0))
        x_test,y_test = x[d],y[d]
    index += 1
    # scale 
    x_train,x_test = normalize(x_train,x_test)
    # reshape  
    x_train,x_test = reshape_x(x_train,x_test,2048)
    # define model 
    model = lstm_model()
    # train 
    history = model.fit(x_train, y_train, epochs = 100, validation_data= (x_test, y_test))
    score, acc = model.evaluate(x_test, y_test)
    all_accuracy.append(acc)
    if index == 3:
        break 

leave one out # person: 0
Epoch 1/100
45/45 [==============================] - 2s 34ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6917 - val_accuracy: 0.5000
Epoch 2/100
45/45 [==============================] - 1s 21ms/step - loss: 0.7310 - accuracy: 0.5553 - val_loss: 0.5319 - val_accuracy: 0.8889
Epoch 3/100
45/45 [==============================] - 1s 20ms/step - loss: 0.7902 - accuracy: 0.6639 - val_loss: 0.4844 - val_accuracy: 0.8611
Epoch 4/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6830 - accuracy: 0.6940 - val_loss: 0.3822 - val_accuracy: 0.8889
Epoch 5/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6650 - accuracy: 0.6975 - val_loss: 0.3642 - val_accuracy: 0.8889
Epoch 6/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6048 - accuracy: 0.7101 - val_loss: 0.3881 - val_accuracy: 0.8889
Epoch 7/100
45/45 [==============================] - 1s 20ms/step - loss: 0.5971 - accuracy: 0.7241 - val_loss: 0.3721 - v

45/45 [==============================] - 1s 20ms/step - loss: 0.2069 - accuracy: 0.9104 - val_loss: 0.3343 - val_accuracy: 0.8611
Epoch 59/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1915 - accuracy: 0.9244 - val_loss: 0.4447 - val_accuracy: 0.8611
Epoch 60/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1721 - accuracy: 0.9174 - val_loss: 0.3751 - val_accuracy: 0.8611
Epoch 61/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1737 - accuracy: 0.9293 - val_loss: 0.7882 - val_accuracy: 0.8611
Epoch 62/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1731 - accuracy: 0.9258 - val_loss: 0.3613 - val_accuracy: 0.8889
Epoch 63/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1800 - accuracy: 0.9202 - val_loss: 0.5713 - val_accuracy: 0.8056
Epoch 64/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1995 - accuracy: 0.9069 - val_loss: 0.6785 - val_accuracy: 0.8889
Epoch 65/100

45/45 [==============================] - 1s 20ms/step - loss: 0.5722 - accuracy: 0.7437 - val_loss: 0.4119 - val_accuracy: 0.8611
Epoch 15/100
45/45 [==============================] - 1s 20ms/step - loss: 0.5249 - accuracy: 0.7619 - val_loss: 0.4431 - val_accuracy: 0.7778
Epoch 16/100
45/45 [==============================] - 1s 20ms/step - loss: 0.4913 - accuracy: 0.7640 - val_loss: 0.3869 - val_accuracy: 0.8056
Epoch 17/100
45/45 [==============================] - 1s 20ms/step - loss: 0.4851 - accuracy: 0.7668 - val_loss: 0.4313 - val_accuracy: 0.8611
Epoch 18/100
45/45 [==============================] - 1s 20ms/step - loss: 0.6890 - accuracy: 0.7696 - val_loss: 0.3953 - val_accuracy: 0.8611
Epoch 19/100
45/45 [==============================] - 1s 20ms/step - loss: 0.4599 - accuracy: 0.7794 - val_loss: 0.3952 - val_accuracy: 0.8333
Epoch 20/100
45/45 [==============================] - 1s 20ms/step - loss: 0.4465 - accuracy: 0.7948 - val_loss: 0.3966 - val_accuracy: 0.8056
Epoch 21/100

45/45 [==============================] - 1s 20ms/step - loss: 0.1521 - accuracy: 0.9293 - val_loss: 0.4165 - val_accuracy: 0.7778
Epoch 72/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1495 - accuracy: 0.9384 - val_loss: 0.4253 - val_accuracy: 0.8056
Epoch 73/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1461 - accuracy: 0.9419 - val_loss: 0.3815 - val_accuracy: 0.8611
Epoch 74/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1462 - accuracy: 0.9405 - val_loss: 0.4002 - val_accuracy: 0.8611
Epoch 75/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1624 - accuracy: 0.9279 - val_loss: 0.3989 - val_accuracy: 0.8333
Epoch 76/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1218 - accuracy: 0.9475 - val_loss: 0.5955 - val_accuracy: 0.7222
Epoch 77/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1374 - accuracy: 0.9489 - val_loss: 0.5423 - val_accuracy: 0.8333
Epoch 78/100

45/45 [==============================] - 1s 20ms/step - loss: 0.5142 - accuracy: 0.8277 - val_loss: 0.9328 - val_accuracy: 0.6667
Epoch 28/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3755 - accuracy: 0.8382 - val_loss: 1.1834 - val_accuracy: 0.6667
Epoch 29/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3673 - accuracy: 0.8291 - val_loss: 1.3046 - val_accuracy: 0.7222
Epoch 30/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3461 - accuracy: 0.8396 - val_loss: 1.0098 - val_accuracy: 0.6667
Epoch 31/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3902 - accuracy: 0.8417 - val_loss: 1.0391 - val_accuracy: 0.6389
Epoch 32/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3513 - accuracy: 0.8445 - val_loss: 1.1579 - val_accuracy: 0.6944
Epoch 33/100
45/45 [==============================] - 1s 20ms/step - loss: 0.3386 - accuracy: 0.8564 - val_loss: 1.3716 - val_accuracy: 0.6389
Epoch 34/100

45/45 [==============================] - 1s 20ms/step - loss: 0.2679 - accuracy: 0.9209 - val_loss: 3.0894 - val_accuracy: 0.6389
Epoch 85/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1047 - accuracy: 0.9636 - val_loss: 4.8286 - val_accuracy: 0.6389
Epoch 86/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1248 - accuracy: 0.9489 - val_loss: 3.1242 - val_accuracy: 0.6944
Epoch 87/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1511 - accuracy: 0.9335 - val_loss: 3.3438 - val_accuracy: 0.6667
Epoch 88/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1155 - accuracy: 0.9594 - val_loss: 3.3547 - val_accuracy: 0.6389
Epoch 89/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0997 - accuracy: 0.9587 - val_loss: 4.3894 - val_accuracy: 0.5556
Epoch 90/100
45/45 [==============================] - 1s 20ms/step - loss: 0.1808 - accuracy: 0.9398 - val_loss: 2.4920 - val_accuracy: 0.5833
Epoch 91/100

In [75]:
print(all_accuracy)
import pandas as pd 
df = pd.DataFrame(all_accuracy)
df.to_csv('test.csv')

[0.8888888955116272, 0.8611111044883728, 0.6111111044883728, 0.7222222089767456, 0.8611111044883728, 0.9166666865348816, 0.7777777910232544, 0.6666666865348816, 0.6944444179534912, 0.6666666865348816, 0.6333333253860474, 0.8333333134651184, 0.8055555820465088, 0.6666666865348816, 0.5555555820465088, 0.6388888955116272, 0.75, 0.6666666865348816, 0.75, 0.5277777910232544, 0.6666666865348816, 0.6388888955116272, 0.4722222089767456, 0.6944444179534912, 0.8333333134651184, 0.5277777910232544, 0.7222222089767456, 0.5833333134651184, 0.6111111044883728, 0.7777777910232544, 0.75, 0.6944444179534912, 0.5555555820465088, 0.7777777910232544, 0.6111111044883728, 0.6666666865348816, 0.6944444179534912, 0.5277777910232544, 0.8333333134651184, 0.6333333253860474, 0.6666666865348816, 0.6000000238418579]


In [140]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 50, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/50
39/39 [==============================] - 2s 58ms/step - loss: 0.7017 - accuracy: 0.5858 - val_loss: 0.5693 - val_accuracy: 0.7319
Epoch 2/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6533 - accuracy: 0.6788 - val_loss: 0.5839 - val_accuracy: 0.7391
Epoch 3/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6793 - accuracy: 0.7087 - val_loss: 0.4724 - val_accuracy: 0.7536
Epoch 4/50
39/39 [==============================] - 1s 35ms/step - loss: 0.5993 - accuracy: 0.7193 - val_loss: 0.4537 - val_accuracy: 0.7754
Epoch 5/50
39/39 [==============================] - 1s 35ms/step - loss: 0.6023 - accuracy: 0.7751 - val_loss: 0.4327 - val_accuracy: 0.7899
Epoch 6/50
39/39 [==============================] - 1s 34ms/step - loss: 0.5284 - accuracy: 0.7896 - val_loss: 0.5186 - val_accuracy: 0.7899
Epoch 7/50
39/39 [==============================] - 1s 34ms/step - loss: 0.4642 - accuracy: 0.8091 - val_loss: 0.5392 - val_accuracy: 0.7464
Epoch 8/50
39

In [ ]:
### leave one out 


In [60]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
# %tensorboard --logdir logs

#### CNN 

In [141]:
## reshape 
x_train = np.reshape(x_train, (x_train.shape[0],new_x.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],new_x.shape[1],1))

In [142]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1236, 2048, 1) (138, 2048, 1)
(1236, 1) (138, 1)


In [143]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed
model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())

model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(3,activation="softmax"))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2039, 16)          176       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1018, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 16288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1628900   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 1,630,163
Trainable params: 1,630,163
Non-trainable params: 0
____________________________________________

In [145]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 30, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/30
39/39 [==============================] - 1s 23ms/step - loss: 0.2049 - accuracy: 0.9231 - val_loss: 0.6175 - val_accuracy: 0.7319
Epoch 2/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1816 - accuracy: 0.9304 - val_loss: 0.6375 - val_accuracy: 0.7319
Epoch 3/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1442 - accuracy: 0.9628 - val_loss: 0.7088 - val_accuracy: 0.7536
Epoch 4/30
39/39 [==============================] - 1s 23ms/step - loss: 0.1179 - accuracy: 0.9636 - val_loss: 0.7885 - val_accuracy: 0.7464
Epoch 5/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1055 - accuracy: 0.9741 - val_loss: 0.9229 - val_accuracy: 0.7536
Epoch 6/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1149 - accuracy: 0.9668 - val_loss: 0.8137 - val_accuracy: 0.7681
Epoch 7/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1165 - accuracy: 0.9604 - val_loss: 0.8468 - val_accuracy: 0.7319
Epoch 8/30
39

#### CRNN

In [96]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed

model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
# model.add(AveragePooling2D((5, 1), strides=(5, 1)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(40, activation="sigmoid", dropout=0.25, return_sequences=False))
model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(2,activation="softmax"))
## rmsprop
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 2039, 16)          176       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1018, 16)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1018, 16)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40)                9120      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_10 (Dense)             (None, 2)               

In [97]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 20, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/20
40/40 [==============================] - 22s 562ms/step - loss: 0.6987 - accuracy: 0.4545 - val_loss: 0.6895 - val_accuracy: 0.5461
Epoch 2/20
40/40 [==============================] - 20s 493ms/step - loss: 0.6962 - accuracy: 0.5028 - val_loss: 0.6881 - val_accuracy: 0.5461
Epoch 3/20
40/40 [==============================] - 18s 438ms/step - loss: 0.6938 - accuracy: 0.5028 - val_loss: 0.6887 - val_accuracy: 0.5745
Epoch 4/20
40/40 [==============================] - 18s 442ms/step - loss: 0.6892 - accuracy: 0.5368 - val_loss: 0.6769 - val_accuracy: 0.5532
Epoch 5/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6792 - accuracy: 0.5558 - val_loss: 0.6721 - val_accuracy: 0.5248
Epoch 6/20
40/40 [==============================] - 17s 436ms/step - loss: 0.6562 - accuracy: 0.6144 - val_loss: 0.6300 - val_accuracy: 0.6738
Epoch 7/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6445 - accuracy: 0.6207 - val_loss: 0.6263 - val_accuracy: 0.6383